In [16]:
import cv2
import sys
import numpy

# Different constants for image processing filters
PREVIEW  = 0  # Preview Mode
BLUR     = 1  # Blurring Filter
FEATURES = 2  # Corner Feature Detector
CANNY    = 3  # Canny Edge Detector

# Used in edge detection
feature_params = dict(maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)

# Using webcam as video source
s = 0
if len(sys.argv) > 1:
    s = sys.argv[1]

image_filter = FEATURES
alive = True

# Open named window to show processed frames
win_name = "Camera Filters"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None

source = cv2.VideoCapture(s)

while alive:
    # If there is a frame and has been read
    has_frame, frame = source.read()
    if not has_frame:
        break

    # flips the frame horizontally
    frame = cv2.flip(frame, 1)

    # shows live feed, doesn't process anything
    if image_filter == PREVIEW:
        result = frame

    # Detects edges by marking pixels where the intensity changes significantly using the OpenCV algorithm
    # 80 and 150 are the thresholds for edge detection for gradients
    # below 80 are considered non-edges, above 150 are string edges
    elif image_filter == CANNY:
        result = cv2.Canny(frame, 80, 150)

    # smooths out variations between neighbouring pixels
    # applies simple averaging filter to the frame, here is 13 x 13 square kernel
    # smaller is lighter blur, larger is more pronounced blurring
    elif image_filter == BLUR:
        result = cv2.blur(frame, (13, 13))

    # detects corners in the frame
    elif image_filter == FEATURES:
        result = frame
        # turns BGR to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # detects corners in grayscale frame using the params in feature params
        corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params)
        # if there are corners, draw green circles around them on the original color frame
        if corners is not None:
            for x, y in numpy.float32(corners).reshape(-1, 2):
                cv2.circle(result, (x, y), 10, (0, 255, 0), 1)

    cv2.imshow(win_name, result)

    # Change modes based on which computer keys are pressed, or kills it when Q or Esc are pressed
    key = cv2.waitKey(1)
    if key == ord("Q") or key == ord("q") or key == 27:
        alive = False
    elif key == ord("C") or key == ord("c"):
        image_filter = CANNY
    elif key == ord("B") or key == ord("b"):
        image_filter = BLUR
    elif key == ord("F") or key == ord("f"):
        image_filter = FEATURES
    elif key == ord("P") or key == ord("p"):
        image_filter = PREVIEW

source.release()
cv2.destroyWindow(win_name)

In [36]:
## MY OWN CODE TO PROCESS STORED VIDEO RATHER THAN VIDEO FEED
import cv2

PREVIEW  = 0
BLUR     = 1
FEATURES = 2
CANNY    = 3

image_filter = BLUR

# maxCorners: max number of corners that can be detected per frame
# qualityLevel: minimal accepted quality of corners, higher is better quality
# minDistance: minimum possible Euclidean distance between the detected corners
# blockSize: size of the window for corner detection, 9 means 9x9 pixel neighbourhood considered for each point
feature_params = dict(maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)

# Source video file
source = r'C:\Users\Work\Documents\projects\opencv-bootcamp\video-writing\race_car.mp4'

# Open the video capture
video_cap = cv2.VideoCapture(source)

# Check if video capture opened
if not video_cap.isOpened():
    print('Error opening video stream or file')

# Get frame dimensions and fps of original video to use for new video
frame_width = int(video_cap.get(3))
frame_height = int(video_cap.get(4))
fps = video_cap.get(cv2.CAP_PROP_FPS)

# Create VideoWriter object, will create new file that has been processed
out_mp4 = cv2.VideoWriter(r'C:\Users\Work\Documents\projects\opencv-bootcamp\image-filtering\race_car_out.mp4', cv2.VideoWriter_fourcc(*'XVID'), 10, (frame_width, frame_height))

while video_cap.isOpened():
    ret, frame = video_cap.read()

    if not ret:
        print('No frame read!')
        break

    if image_filter == PREVIEW:
        result = frame
    elif image_filter == CANNY:
        edges = cv2.Canny(frame, 80, 150)
        result = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    elif image_filter == BLUR:
        result = cv2.blur(frame, (30, 30))
    elif image_filter == FEATURES:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params)
        result = frame.copy()
        print(result)
        for corner in corners:
            x, y = corner.ravel()
            cv2.circle(result, (int(x), int(y)), 12, (0, 0, 255), 3)
    else:
        break

    out_mp4.write(result)

# Release video captures and writers
video_cap.release()
out_mp4.release()


No frame read!
